<a href="https://colab.research.google.com/github/TomasMendozaHN/ComputerVision_Networks/blob/master/CorrelationMatrix_ConfusionMatrix_FeatureDistribution.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import files
uploaded = files.upload()

In [ ]:
!mkdir ~/.kaggle #mkdir = make directory
!cp kaggle.json ~/.kaggle/  # cp = copy
! chmod 600 ~/.kaggle/kaggle.json
! kaggle competitions download titanic

In [ ]:
!unzip titanic.zip

In [ ]:
import pandas as pd
trainData = pd.read_csv("train.csv")
trainData.head()

In [171]:
# Fill NaN values with 0
trainData = trainData.fillna(0)

# Drop useless data
trainData = trainData.drop(['Name', 'Ticket'], axis='columns')

In [ ]:
trainData.head()

# Visualize Data

In [ ]:
trainData.hist(figsize=(16,8))

In [ ]:
trainData.groupby('Survived').hist(figsize=(16,8))

# Visualize Correlations

In [ ]:
import matplotlib.pyplot as plt
plt.rcParams['figure.figsize']=(18,14)
import seaborn as sns
corrMatrix = trainData.corr()
sns.heatmap(corrMatrix, annot=True)

plt.show()

# Convert Numerical to Categorical

In [ ]:
trainData = pd.get_dummies(trainData)
trainData.head()

In [ ]:
trainY = trainData['Survived']
trainX = trainData.drop(['Survived'], axis='columns')
trainX, trainY

In [ ]:
# Convert dataframes to numpy arrays
import numpy as np
trainX, trainY = np.array(trainX), np.array(trainY)
trainX.shape, trainY.shape

In [179]:
from sklearn.utils import shuffle
trainX, trainY = shuffle(trainX, trainY, random_state=0)

In [ ]:
from sklearn.model_selection import train_test_split
trainX, testX, trainY, testY = train_test_split(trainX, trainY, test_size=0.30, random_state=42)
trainX.shape, testX.shape

# Define MLP model

In [181]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch.utils.data as data
from torch.nn import LogSoftmax

class MLP(nn.Module):
    def __init__(self, input_dim, output_dim):
        super().__init__()

        self.input_fc = nn.Linear(input_dim, 128)
        self.hidden_fc = nn.Linear(128, 128)
        self.hidden_fc2 = nn.Linear(128, 128)
        self.hidden_fc3 = nn.Linear(128, 128)
        self.hidden_fc4 = nn.Linear(128, 128)
        self.hidden_fc5 = nn.Linear(128, 128)
        self.output_fc6 = nn.Linear(128, output_dim)

    def forward(self, x):
        h_1 = F.relu(self.input_fc(x))
        h_2 = F.relu(self.hidden_fc(h_1))
        h_3 = F.relu(self.hidden_fc2(h_2))
        h_4 = F.relu(self.hidden_fc3(h_3))
        h_5 = F.relu(self.hidden_fc4(h_4))
        h_6 = F.relu(self.hidden_fc5(h_5))
        y_pred = self.output_fc6(h_6)
        return y_pred

In [182]:
net = MLP(160,2)

In [183]:
import torch.optim as optim
# initialize our optimizer and loss function
loss_function = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(net.parameters(), lr=1e-4)

In [ ]:
import tqdm

hist_loss = []

for epoch in tqdm.tqdm(range(2000)):

  optimizer.zero_grad()

  X = torch.from_numpy(trainX).type(torch.FloatTensor)
  Y = torch.from_numpy(trainY).type(torch.LongTensor)

  outputs = net(X)

  loss = loss_function(outputs, Y)

  loss.backward()

  optimizer.step()

  hist_loss.append(loss.detach().cpu().numpy())



In [ ]:
plt.rcParams['figure.figsize'] = (14,6)
plt.plot(hist_loss)
plt.show()

# Test!

In [186]:
def calculate_accuracy(y_pred, y):
    top_pred = y_pred.argmax(1, keepdim=True)
    correct = top_pred.eq(y.view_as(top_pred)).sum()
    acc = correct.float() / y.shape[0]
    return acc

In [ ]:
net.eval()

X = torch.from_numpy(testX).type(torch.FloatTensor)
Y = torch.from_numpy(testY).type(torch.LongTensor)

y_pred = net(X)

probs = F.softmax(y_pred, dim=-1)
pred_labels = torch.argmax(probs, 1).detach().numpy()

loss = loss_function(y_pred, Y)

acc = calculate_accuracy(y_pred, Y)
acc = float(acc.detach().numpy())
print(f"The accuracy of your model is = {round(acc,4)*100}%")

# Correlation Matrix

In [261]:
from sklearn import metrics

def plot_confusion_matrix(labels, pred_labels):

    fig = plt.figure(figsize=(10, 10))
    ax = fig.add_subplot(1, 1, 1)
    cm = metrics.confusion_matrix(labels, pred_labels)
    cm = metrics.ConfusionMatrixDisplay(cm, display_labels=range(10))
    cm.plot(values_format='d', cmap='Blues', ax=ax)

In [ ]:
plot_confusion_matrix(testY, pred_labels)